In [ ]:
import os
os.chdir("..")

import midi_np_translation.midi2input as midi2input
import numpy as np

In [ ]:
input_output_directories = [
    ("input_midi/Jazz_ML_MIDI/Jazz Midi/", "preprocessed_dataset/Jazz_ML_MIDI/"), 
    ("input_midi/jazz_standards/", "preprocessed_dataset/jazz_standards/")
]

if not os.path.exists("preprocessed_dataset/Jazz_ML_MIDI/"):
    os.mkdir("preprocessed_dataset/Jazz_ML_MIDI/")
if not os.path.exists("preprocessed_dataset/jazz_standards/"):
    os.mkdir("preprocessed_dataset/jazz_standards/")

### run in sequence

In [ ]:
def run_in_sequence():
    for input_midi_dir, output_np_dir in input_output_directories:
        for midi_file_name in os.listdir(input_midi_dir):
        # print(midi_file_name)
            if not midi_file_name.lower().endswith(".mid"):
                continue
            try:
                data_rows, bass_notes = midi2input.midi_to_input(input_midi_dir + midi_file_name)
            except AssertionError as e:
                print(f"'{midi_file_name}' AssertionError: ", end="")
                print(e)
            except Exception as e:
                print(f"'{midi_file_name}' Error: ", end="")
                print(e)
            else:
                np.save(output_np_dir + midi_file_name + ".npy", data_rows)
                np.save(output_np_dir + midi_file_name + ".ans.npy", bass_notes)

# run_in_sequence()

### run in parallel

In [ ]:
def process_a_single_midi_file(args):
    input_midi_dir, output_np_dir, midi_file_name = args
    if not midi_file_name.lower().endswith(".mid"):
        return
    try:
        data_rows, bass_notes = midi2input.midi_to_input(input_midi_dir + midi_file_name)
    except AssertionError as e:
        print(f"AssertionError '{midi_file_name}': ", end="")
        print(e)
    except Exception as e:
        print(f"Error '{midi_file_name}': ", end="")
        print(e)
    else:
        np.save(output_np_dir + midi_file_name + ".npy", data_rows)
        np.save(output_np_dir + midi_file_name + ".ans.npy", bass_notes)
        print(len(os.listdir(output_np_dir)))

In [ ]:
import multiprocess

def run_in_parallel(input_output_directories):
    all_args = []
    for input_midi_dir, output_np_dir in input_output_directories:
        for midi_file_name in os.listdir(input_midi_dir):
            all_args.append((input_midi_dir, output_np_dir, midi_file_name))

    pool = multiprocess.Pool(3)
    _ = pool.map(process_a_single_midi_file, all_args)

run_in_parallel(input_output_directories = [
    ("input_midi/Jazz_ML_MIDI/Jazz Midi/", "preprocessed_dataset/Jazz_ML_MIDI/"), 
    ("input_midi/jazz_standards/", "preprocessed_dataset/jazz_standards/")
])

In [ ]:
run_in_parallel(input_output_directories = [
    ("input_midi/Jazz_ML_MIDI/Jazz Midi/", "preprocessed_dataset/Jazz_ML_MIDI/"), 
])

In [ ]:
run_in_parallel(input_output_directories = [
    ("input_midi/irealpro_transposed/", "preprocessed_dataset/irealpro_midi/"), 
])